<a href="https://colab.research.google.com/github/Saksham-tyagi1/NBA-Analysis/blob/main/NBAFeatureEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler,PowerTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',100)
pd.set_option("display.max_colwidth", 1)

In [ ]:
!pip install category_encoders

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/nba_shot_data.parquet')

In [ ]:
df

,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,POSITION_GROUP,POSITION,GAME_DATE,GAME_ID,HOME_TEAM,AWAY_TEAM,SHOT_MADE,ACTION_TYPE,SHOT_TYPE,BASIC_ZONE,ZONE_NAME,ZONE_RANGE,LOC_X,LOC_Y,SHOT_DISTANCE,QUARTER,MINS_LEFT,SECS_LEFT,SPD,DIST,ORBC,DRBC,TCHS,SAST,AST,FGM,FGA,DFGM,DFGA,DFG_PCT,SHOT_CLOCK_RANGE,SCORE,SCOREMARGIN,EFG_PCT,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,LOC_X_BIN,LOC_Y_BIN,SHOT_DISTANCE_BIN,ACTION_TYPE_modified,game_time
0,1610612764,Washington Wizards,1630557,Corey Kispert,F,SF,03-16-2024,22300971,CHI,WAS,False,Turnaround Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center,8-16 ft.,0.1,18.049999,12.0,1,11,42,4.63,2.37,0.0,4.0,44.0,1.0,1.0,380.0,785.0,2.0,6.0,0.333,22-18 Very Early,0 - 2,2.0,0.599,4.0,0.0,0.0,"(0.0, 2.5]","(16.93, 22.49]",midrange,Other,0.30
1,1610612764,Washington Wizards,1630557,Corey Kispert,F,SF,03-16-2024,22300971,CHI,WAS,False,Turnaround Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center,8-16 ft.,0.1,18.049999,12.0,1,11,42,4.63,2.37,0.0,4.0,44.0,1.0,1.0,380.0,785.0,2.0,6.0,0.333,22-18 Very Early,3 - 2,-1.0,0.599,5.0,5.0,0.0,"(0.0, 2.5]","(16.93, 22.49]",midrange,Other,0.30
2,1610612764,Washington Wizards,1630557,Corey Kispert,F,SF,03-16-2024,22300971,CHI,WAS,False,Turnaround Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center,8-16 ft.,0.1,18.049999,12.0,1,11,42,4.63,2.37,0.0,4.0,44.0,1.0,1.0,380.0,785.0,2.0,6.0,0.333,22-18 Very Early,3 - 5,2.0,0.599,4.0,4.0,0.0,"(0.0, 2.5]","(16.93, 22.49]",midrange,Other,0.30
3,1610612764,Washington Wizards,1630557,Corey Kispert,F,SF,03-16-2024,22300971,CHI,WAS,False,Turnaround Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center,8-16 ft.,0.1,18.049999,12.0,1,11,42,4.63,2.37,0.0,4.0,44.0,1.0,1.0,380.0,785.0,2.0,6.0,0.333,22-18 Very Early,6 - 5,-1.0,0.599,5.0,0.0,0.0,"(0.0, 2.5]","(16.93, 22.49]",midrange,Other,0.30
4,1610612764,Washington Wizards,1630557,Corey Kispert,F,SF,03-16-2024,22300971,CHI,WAS,False,Turnaround Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center,8-16 ft.,0.1,18.049999,12.0,1,11,42,4.63,2.37,0.0,4.0,44.0,1.0,1.0,380.0,785.0,2.0,6.0,0.333,22-18 Very Early,6 - 7,1.0,0.599,4.0,0.0,0.0,"(0.0, 2.5]","(16.93, 22.49]",midrange,Other,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20134689,1610612737,Atlanta Hawks,1628981,Bruno Fernando,C,C,04-10-2024,22301159,ATL,CHA,False,Layup Shot,2PT Field Goal,Restricted Area,Center,Less Than 8 ft.,1.1,8.450000,3.0,4,0,9,3.91,1.74,0.0,9.0,41.0,0.0,0.0,3.0,11.0,5.0,6.0,0.833,4-0 Very Late,111 - 108,-3.0,0.273,5.0,0.0,0.0,"(0.0, 2.5]","(5.81, 11.37]",layup,Layup Shot,47.85
20134690,1610612737,Atlanta Hawks,1628981,Bruno Fernando,C,C,04-10-2024,22301159,ATL,CHA,False,Layup Shot,2PT Field Goal,Restricted Area,Center,Less Than 8 ft.,1.1,8.450000,3.0,4,0,9,3.91,1.74,0.0,9.0,41.0,0.0,0.0,3.0,11.0,5.0,6.0,0.833,4-0 Very Late,111 - 110,-1.0,0.273,4.0,0.0,0.0,"(0.0, 2.5]","(5.81, 11.37]",layup,Layup Shot,47.85
20134691,1610612737,Atlanta Hawks,1628981,Bruno Fernando,C,C,04-10-2024,22301159,ATL,CHA,False,Layup Shot,2PT Field Goal,Restricted Area,Center,Less Than 8 ft.,1.1,8.450000,3.0,4,0,9,3.91,1.74,0.0,9.0,41.0,0.0,0.0,3.0,11.0,5.0,6.0,0.833,4-0 Very Late,111 - 112,1.0,0.273,4.0,0.0,0.0,"(0.0, 2.5]","(5.81, 11.37]",layup,Layup Shot,47.85
20134692,1610612737,Atlanta Hawks,1628981,Bruno Fernando,C,C,04-10-2024,22301159,ATL,CHA,False,Layup Shot,2PT Field Goal,Restricted Area,Center,Less Than 8 ft.,1.1,8.450000,3.0,4,0,9,3.91,1.74,0.0,9.0,41.0,0.0,0.0,3.0,11.0,5.0,6.0,0.833,4-0 Very Late,111 - 114,3.0,0.273,4.0,0.0,0.0,"(0.0, 2.5]","(5.81, 11.37]",layup,Layup Shot,47.85


In [ ]:
df['SCOREMARGIN'].fillna(0, inplace=True)


<ipython-input-6-610f65e7b0e5>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SCOREMARGIN'].fillna(0, inplace=True)


In [ ]:
df_encoded = df

# Utility Functions


In [ ]:
def one_hot_encode(df, columns, drop_first=False, prefix=None):
    encoded_df = pd.get_dummies(df[columns], drop_first=drop_first, prefix=prefix)
    df_remaining = df.drop(columns=columns)
    return pd.concat([df_remaining, encoded_df], axis=1)


def label_encode(df, columns, inplace=False):
    label_encoder = LabelEncoder()

    if inplace:
        # Modify `df` in place
        for col in columns:
            if col in df.columns:
                df[col] = label_encoder.fit_transform(df[col])
            else:
                raise ValueError(f"Column '{col}' not found in DataFrame.")
        return df
    else:
        # Create a copy and apply label encoding
        new_df = df.copy()
        for col in columns:
            if col in new_df.columns:
                new_df[col] = label_encoder.fit_transform(new_df[col])
            else:
                raise ValueError(f"Column '{col}' not found in DataFrame.")
        return new_df
def frequency_encode(df, columns):
    for col in columns:
        # Check if the frequency-encoded column already exists to avoid overwriting
        freq_col = f"{col}_freq"
        if freq_col in df.columns:
            raise ValueError(f"Column '{freq_col}' already exists in DataFrame.")

        # Calculate frequency encoding
        freq_encoding = df[col].value_counts().to_dict()
        df[freq_col] = df[col].map(freq_encoding)

    return df


In [ ]:
# def one_hot_encode(df, columns, inplace=False, drop_first=False):
#     encoded_df = pd.get_dummies(df[columns], drop_first=drop_first)

#     if inplace:
#         df.drop(columns=columns, inplace=True)
#         df = pd.concat([df, encoded_df], axis=1)
#         return df
#     else:
#         return pd.concat([df.drop(columns=columns), encoded_df], axis=1)
# def label_encode(df, columns, inplace=False):
#     label_encoder = LabelEncoder()
#     if inplace:
#         for col in columns:
#             if col in df.columns:
#                 df[col] = label_encoder.fit_transform(df[col])
#             else:
#                 raise ValueError(f"Column '{col}' not found in DataFrame.")
#         return df
#     else:
#         new_df = df.copy()
#         for col in columns:
#             if col in new_df.columns:
#                 new_df[col] = label_encoder.fit_transform(new_df[col])
#             else:
#                 raise ValueError(f"Column '{col}' not found in DataFrame.")
#         return new_df
# def frequency_encode(df, columns):
#     for col in columns:
#         freq_encoding = df[col].value_counts().to_dict()
#         df[f"{col}_freq"] = df[col].map(freq_encoding)
#     return df



In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

def scale_features(df, features):
    # Initialize scalers
    scalers = {
        'StandardScaler': StandardScaler(),
        'MinMaxScaler': MinMaxScaler(),
        'RobustScaler': RobustScaler()
    }

    # Initialize a dictionary to store scaler suggestions
    scaler_suggestions = {}

    # Copy the dataframe to avoid modifying the original
    df_scaled = df.copy()

    for feature in features:
        # Calculate skewness
        skewness = df[feature].skew()
        # Calculate the number of outliers using the IQR method
        q1 = df[feature].quantile(0.25)
        q3 = df[feature].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)][feature]
        num_outliers = outliers.count()

        # Decide which scaler to use
        if abs(skewness) < 0.5 and num_outliers == 0:
            scaler_type = 'StandardScaler'
        elif abs(skewness) > 1 or num_outliers > 0.05 * len(df):
            scaler_type = 'RobustScaler'
        else:
            scaler_type = 'MinMaxScaler'

        # Store the suggestion
        scaler_suggestions[feature] = {
            'Skewness': skewness,
            'Outliers': num_outliers,
            'Scaler': scaler_type
        }

        # Apply the scaler
        scaler = scalers[scaler_type]
        df_scaled[[feature]] = scaler.fit_transform(df[[feature]])

    # Convert suggestions to a DataFrame for easy viewing
    suggestions_df = pd.DataFrame(scaler_suggestions).T
    print("\nScaler Suggestions:\n", suggestions_df)

    return df_scaled, suggestions_df



In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import pandas as pd

def scale_features_based_on_distribution(df, features):
    """
    Scales features based on their distribution:
    - StandardScaler for normally distributed features with few outliers
    - MinMaxScaler for moderately skewed features or those requiring fixed range [0, 1]
    - RobustScaler for features with significant outliers or high skewness

    Parameters:
        df (pd.DataFrame): DataFrame containing the features to be scaled.
        features (list): List of feature names to be scaled.

    Returns:
        df_scaled (pd.DataFrame): DataFrame with scaled features.
        scaler_suggestions (pd.DataFrame): DataFrame showing scaling choices for each feature.
    """
    # Initialize scalers
    scalers = {
        'StandardScaler': StandardScaler(),
        'MinMaxScaler': MinMaxScaler(),
        'RobustScaler': RobustScaler()
    }

    scaler_suggestions = {}  # To store scaler decisions
    df_scaled = df.copy()  # Copy to avoid modifying original data

    for feature in features:
        # Calculate skewness and IQR-based outliers
        skewness = df[feature].skew()
        q1 = df[feature].quantile(0.25)
        q3 = df[feature].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        num_outliers = ((df[feature] < lower_bound) | (df[feature] > upper_bound)).sum()

        # Decide on the scaler based on skewness and outliers
        if abs(skewness) < 0.5 and num_outliers == 0:
            scaler_type = 'StandardScaler'
        elif abs(skewness) > 1 or num_outliers > 0.05 * len(df):
            scaler_type = 'RobustScaler'
        else:
            scaler_type = 'MinMaxScaler'

        # Apply the chosen scaler
        scaler = scalers[scaler_type]
        df_scaled[[feature]] = scaler.fit_transform(df[[feature]])

        # Store the decision in suggestions dictionary
        scaler_suggestions[feature] = {
            'Skewness': skewness,
            'Outliers': num_outliers,
            'Selected Scaler': scaler_type
        }

    # Convert scaler suggestions to DataFrame for easy viewing
    scaler_suggestions_df = pd.DataFrame(scaler_suggestions).T
    print("\nScaler Suggestions:\n", scaler_suggestions_df)

    return df_scaled, scaler_suggestions_df


In [ ]:
df_encoded = df

# Feature Engineering

In [ ]:
# Convert 'MINS_LEFT' and 'SECS_LEFT' to int16 if not already done
df_encoded['MINS_LEFT'] = df_encoded['MINS_LEFT'].astype('int16')
df_encoded['SECS_LEFT'] = df_encoded['SECS_LEFT'].astype('int16')

# Calculate 'time_left_in_quarter' and set to int32 to prevent overflow
df_encoded['time_left_in_quarter'] = (df_encoded['MINS_LEFT'] * 60 + df_encoded['SECS_LEFT']).astype('int32')

# 1. Calculate time left in the game based on quarter and time left in the quarter
df_encoded['time_left_in_game'] = df_encoded.apply(
    lambda row: ((4 - row['QUARTER']) * 12 * 60 + row['time_left_in_quarter']) if row['QUARTER'] <= 4
                else (5 * 60 - row['time_left_in_quarter']) if row['QUARTER'] > 4
                else None,
    axis=1
)

# 2. Identify the last shot of the quarter
df_encoded['last_shot_of_quarter'] = (df_encoded['time_left_in_quarter'] <= 2).astype(int)

shot_clock_map = {
    '22-18 Very Early': 'Very Early',
    '18-15 Early': 'Early',
    '15-7 Average': 'Average',
    '7-4 Late': 'Late',
    '4-0 Very Late': 'Very Late'
}
df_encoded['shot_clock_status'] = df_encoded['SHOT_CLOCK_RANGE'].map(shot_clock_map).fillna('Unknown')

# 4. Define clutch moments: last 5 minutes of the game with a score margin of 5 points or fewer
df_encoded['is_clutch'] = ((df_encoded['time_left_in_game'] <= 5 * 60) & (df_encoded['SCOREMARGIN'].abs() <= 5)).astype(int)

# 5. Sort values to apply rolling calculations for each player and game
df_encoded = df_encoded.sort_values(by=['PLAYER_ID', 'GAME_ID', 'game_time'])

# 6. Calculate rolling effective field goal percentage over the last 5 shots
df_encoded['rolling_efg_pct'] = df_encoded.groupby(['PLAYER_ID', 'GAME_ID'])['EFG_PCT'].transform(lambda x: x.rolling(5, min_periods=1).mean())

# 7. Calculate rolling score margin over the last 5 events in each game
df_encoded['rolling_score_margin'] = df_encoded.groupby(['GAME_ID'])['SCOREMARGIN'].transform(lambda x: x.rolling(5, min_periods=1).mean())

# 8. Identify shots taken in the last 30 seconds of the quarter
df_encoded['last_30_seconds'] = (df_encoded['time_left_in_quarter'] <= 30).astype(int)

# 9. Count shots taken in the last 30 seconds within each quarter for each player
df_encoded['shots_in_last_30_seconds'] = df_encoded.groupby(['PLAYER_ID', 'QUARTER'])['last_30_seconds'].transform('sum')

# Optional: Display created features to verify
print(df_encoded[['time_left_in_game', 'last_shot_of_quarter', 'shot_clock_status',
                  'is_clutch', 'rolling_efg_pct', 'rolling_score_margin',
                  'last_30_seconds', 'shots_in_last_30_seconds']].head())


          time_left_in_game  last_shot_of_quarter shot_clock_status  \
10576410  2672               0                     Very Early         
10576411  2672               0                     Very Early         
10576412  2672               0                     Very Early         
10576413  2672               0                     Very Early         
10576414  2672               0                     Very Early         

          is_clutch  rolling_efg_pct  rolling_score_margin  last_30_seconds  \
10576410  0          0.596            3.000000              0                 
10576411  0          0.596            2.000000              0                 
10576412  0          0.596            2.333333              0                 
10576413  0          0.596            2.000000              0                 
10576414  0          0.596            2.200000              0                 

          shots_in_last_30_seconds  
10576410  0                         
10576411  0             

In [ ]:
df_encoded['quarter_progress'] = pd.cut(df_encoded['time_left_in_quarter'],
                                        bins=[0, 240, 480, 720],
                                        labels=['Early', 'Middle', 'Late'],
                                        right=False,
                                        include_lowest=True)


In [ ]:
df_encoded['quarter_progress'].value_counts()

,count
quarter_progress,
Early,6955149
Middle,6669072
Late,6510473


In [ ]:
df_encoded['SHOT_ATTEMPT'] = 1
df_encoded['zone_combination'] = df_encoded['BASIC_ZONE'] + '_' + df_encoded['ZONE_NAME']
zone_combination_stats = df_encoded.groupby(['PLAYER_NAME', 'zone_combination']).agg(
    zone_combination_attempts=('SHOT_ATTEMPT', 'sum'),
    zone_combination_made=('SHOT_MADE', 'sum')
).reset_index()
zone_combination_stats['zone_combination_success_rate'] = (
    zone_combination_stats['zone_combination_made'] / zone_combination_stats['zone_combination_attempts']
)
df_encoded = df_encoded.merge(zone_combination_stats, on=['PLAYER_NAME', 'zone_combination'], how='left')
basic_zone_freq = df_encoded['BASIC_ZONE'].value_counts(normalize=True).to_dict()
df_encoded['BASIC_ZONE_freq'] = df_encoded['BASIC_ZONE'].map(basic_zone_freq)

In [ ]:

# 1. Zone Defense Profile
zone_defense = df_encoded.groupby(['PLAYER_ID', 'BASIC_ZONE']).agg(
    zone_dfgm=('DFGM', 'sum'),
    zone_dfga=('DFGA', 'sum')
).reset_index()
zone_defense['zone_dfg%'] = zone_defense['zone_dfgm'] / zone_defense['zone_dfga'].replace(0, np.nan)

# Drop existing 'zone_dfg%' in df_encoded if it exists to avoid merge conflict
df_encoded = df_encoded.drop(columns=['zone_dfg%'], errors='ignore')

# Merge back to main DataFrame
df_encoded = df_encoded.merge(zone_defense[['PLAYER_ID', 'BASIC_ZONE', 'zone_dfg%']], on=['PLAYER_ID', 'BASIC_ZONE'], how='left')
df_encoded['zone_dfg%'].fillna(0, inplace=True)

# Define DFG% column directly on df_encoded for overall defensive FG%
df_encoded['DFG%'] = df_encoded['DFGM'] / df_encoded['DFGA'].replace(0, np.nan)
df_encoded['DFG%'].fillna(0, inplace=True)

# 2. Shot Selection Profile by Zone
shot_selection_by_zone = df_encoded.groupby(['PLAYER_ID', 'BASIC_ZONE']).agg(
    shot_attempts=('FGA', 'sum'),
    shots_made=('FGM', 'sum'),
).reset_index()
shot_selection_by_zone['zone_fg%'] = shot_selection_by_zone['shots_made'] / shot_selection_by_zone['shot_attempts'].replace(0, np.nan)

# Drop existing 'zone_fg%' in df_encoded if it exists to avoid merge conflict
df_encoded = df_encoded.drop(columns=['zone_fg%'], errors='ignore')

# Merge this back into the main DataFrame
df_encoded = df_encoded.merge(shot_selection_by_zone[['PLAYER_ID', 'BASIC_ZONE', 'zone_fg%']], on=['PLAYER_ID', 'BASIC_ZONE'], how='left')
df_encoded['zone_fg%'].fillna(0, inplace=True)

# 3. Clutch Shooting Efficiency
df_encoded['is_clutch'] = (df_encoded['time_left_in_game'] <= 300) & (df_encoded['SCOREMARGIN'].abs() <= 5)

# Calculate clutch eFG% for each player
clutch_shooting = df_encoded[df_encoded['is_clutch']].groupby('PLAYER_ID')['EFG_PCT'].mean().reset_index(name='clutch_efg_pct')

# Drop existing 'clutch_efg_pct' in df_encoded if it exists to avoid merge conflict
df_encoded = df_encoded.drop(columns=['clutch_efg_pct'], errors='ignore')

# Merge clutch shooting efficiency back into the main DataFrame
df_encoded = df_encoded.merge(clutch_shooting, on='PLAYER_ID', how='left')
df_encoded['clutch_efg_pct'].fillna(0, inplace=True)

# Clutch defensive DFG%
clutch_defense = df_encoded[df_encoded['is_clutch']].groupby('PLAYER_ID').apply(
    lambda x: pd.Series({'clutch_dfg_pct': x['DFGM'].sum() / x['DFGA'].sum() if x['DFGA'].sum() > 0 else 0})
).reset_index()

# Drop existing 'clutch_dfg_pct' in df_encoded if it exists to avoid merge conflict
df_encoded = df_encoded.drop(columns=['clutch_dfg_pct'], errors='ignore')

# Merge clutch defense efficiency back into the main DataFrame
df_encoded = df_encoded.merge(clutch_defense, on='PLAYER_ID', how='left')
df_encoded['clutch_dfg_pct'].fillna(0, inplace=True)

# 4. Hot Zone Identification
hot_zones = df_encoded.groupby(['PLAYER_ID', 'BASIC_ZONE']).agg(
    shot_attempts=('FGA', 'sum'),
    shots_made=('FGM', 'sum')
).reset_index()
hot_zones['hot_zone_pct'] = hot_zones['shots_made'] / hot_zones['shot_attempts'].replace(0, np.nan)
hot_zones['is_hot_zone'] = hot_zones['hot_zone_pct'] > 0.5  # Adjust threshold as needed

# Drop existing 'is_hot_zone' in df_encoded if it exists to avoid merge conflict
df_encoded = df_encoded.drop(columns=['is_hot_zone'], errors='ignore')

# Merge hot zones back into main DataFrame
df_encoded = df_encoded.merge(hot_zones[['PLAYER_ID', 'BASIC_ZONE', 'is_hot_zone']], on=['PLAYER_ID', 'BASIC_ZONE'], how='left')
df_encoded['is_hot_zone'].fillna(False, inplace=True)

# 5. Interaction Feature: Score Margin x Shot Distance
df_encoded['score_margin_shot_distance'] = df_encoded['SCOREMARGIN'] * df_encoded['SHOT_DISTANCE']

# 6. Rolling EFG% and DFG%
# Rolling DFG% for defenders
df_encoded['rolling_dfg%'] = df_encoded.groupby('PLAYER_ID')['DFG%'].transform(lambda x: x.rolling(window=10, min_periods=1).mean())
df_encoded['rolling_dfg%'].fillna(0, inplace=True)

# Rolling EFG% for shooters
df_encoded['rolling_efg%'] = df_encoded.groupby('PLAYER_ID')['EFG_PCT'].transform(lambda x: x.rolling(window=10, min_periods=1).mean())
df_encoded['rolling_efg%'].fillna(0, inplace=True)

# Display the updated DataFrame
df_encoded.head()


<ipython-input-17-266103203ed2>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_encoded['zone_dfg%'].fillna(0, inplace=True)
<ipython-input-17-266103203ed2>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,POSITION_GROUP,POSITION,GAME_DATE,GAME_ID,HOME_TEAM,AWAY_TEAM,SHOT_MADE,ACTION_TYPE,SHOT_TYPE,BASIC_ZONE,ZONE_NAME,ZONE_RANGE,LOC_X,LOC_Y,SHOT_DISTANCE,QUARTER,MINS_LEFT,SECS_LEFT,SPD,DIST,ORBC,DRBC,TCHS,SAST,AST,FGM,FGA,DFGM,DFGA,DFG_PCT,SHOT_CLOCK_RANGE,SCORE,SCOREMARGIN,EFG_PCT,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,LOC_X_BIN,LOC_Y_BIN,SHOT_DISTANCE_BIN,ACTION_TYPE_modified,game_time,time_left_in_quarter,time_left_in_game,last_shot_of_quarter,shot_clock_status,is_clutch,rolling_efg_pct,rolling_score_margin,last_30_seconds,shots_in_last_30_seconds,quarter_progress,SHOT_ATTEMPT,zone_combination,zone_combination_attempts,zone_combination_made,zone_combination_success_rate,BASIC_ZONE_freq,zone_dfg%,DFG%,zone_fg%,clutch_efg_pct,clutch_dfg_pct,is_hot_zone,score_margin_shot_distance,rolling_dfg%,rolling_efg%
0,1610612747,Los Angeles Lakers,2544,LeBron James,F,PF,03-16-2024,22300973,LAL,GSW,True,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side,8-16 ft.,13.7,7.45,13.0,1,8,32,3.7,2.48,4.0,6.0,94.0,1.0,9.0,664.0,1234.0,2.0,4.0,0.5,22-18 Very Early,0 - 3,3.0,0.596,4.0,4.0,0.0,"(12.5, 15.0]","(5.81, 11.37]",midrange,Turnaround Fadeaway shot,3.466667,512,2672,0,Very Early,False,0.596,3.000000,0,0,Late,1,Mid-Range_Left Side,6888,2646,0.384146,0.112099,0.614779,0.5,0.538118,0.566,0.596584,True,39.0,0.5,0.596
1,1610612747,Los Angeles Lakers,2544,LeBron James,F,PF,03-16-2024,22300973,LAL,GSW,True,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side,8-16 ft.,13.7,7.45,13.0,1,8,32,3.7,2.48,4.0,6.0,94.0,1.0,9.0,664.0,1234.0,2.0,4.0,0.5,22-18 Very Early,2 - 3,1.0,0.596,5.0,5.0,0.0,"(12.5, 15.0]","(5.81, 11.37]",midrange,Turnaround Fadeaway shot,3.466667,512,2672,0,Very Early,False,0.596,2.000000,0,0,Late,1,Mid-Range_Left Side,6888,2646,0.384146,0.112099,0.614779,0.5,0.538118,0.566,0.596584,True,13.0,0.5,0.596
2,1610612747,Los Angeles Lakers,2544,LeBron James,F,PF,03-16-2024,22300973,LAL,GSW,True,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side,8-16 ft.,13.7,7.45,13.0,1,8,32,3.7,2.48,4.0,6.0,94.0,1.0,9.0,664.0,1234.0,2.0,4.0,0.5,22-18 Very Early,2 - 5,3.0,0.596,4.0,4.0,0.0,"(12.5, 15.0]","(5.81, 11.37]",midrange,Turnaround Fadeaway shot,3.466667,512,2672,0,Very Early,False,0.596,2.333333,0,0,Late,1,Mid-Range_Left Side,6888,2646,0.384146,0.112099,0.614779,0.5,0.538118,0.566,0.596584,True,39.0,0.5,0.596
3,1610612747,Los Angeles Lakers,2544,LeBron James,F,PF,03-16-2024,22300973,LAL,GSW,True,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side,8-16 ft.,13.7,7.45,13.0,1,8,32,3.7,2.48,4.0,6.0,94.0,1.0,9.0,664.0,1234.0,2.0,4.0,0.5,22-18 Very Early,4 - 5,1.0,0.596,5.0,0.0,0.0,"(12.5, 15.0]","(5.81, 11.37]",midrange,Turnaround Fadeaway shot,3.466667,512,2672,0,Very Early,False,0.596,2.000000,0,0,Late,1,Mid-Range_Left Side,6888,2646,0.384146,0.112099,0.614779,0.5,0.538118,0.566,0.596584,True,13.0,0.5,0.596
4,1610612747,Los Angeles Lakers,2544,LeBron James,F,PF,03-16-2024,22300973,LAL,GSW,True,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side,8-16 ft.,13.7,7.45,13.0,1,8,32,3.7,2.48,4.0,6.0,94.0,1.0,9.0,664.0,1234.0,2.0,4.0,0.5,22-18 Very Early,4 - 7,3.0,0.596,4.0,0.0,0.0,"(12.5, 15.0]","(5.81, 11.37]",midrange,Turnaround Fadeaway shot,3.466667,512,2672,0,Very Early,False,0.596,2.200000,0,0,Late,1,Mid-Range_Left Side,6888,2646,0.384146,0.112099,0.614779,0.5,0.538118,0.566,0.596584,True,39.0,0.5,0.596


# Encoding



In [ ]:
categorical_features = df.select_dtypes(include=['object', 'category']).columns
cat_feature_analysis = {}
for feature in categorical_features:
    unique_count = df[feature].nunique()
    value_counts = df[feature].value_counts()
    cat_feature_analysis[feature] = {
        'Unique Count': unique_count,
        'Value Counts': value_counts
    }
for feature, analysis in cat_feature_analysis.items():
    print(f"Feature: {feature}")
    print(f"Unique Count: {analysis['Unique Count']}")
    print("Value Counts:")
    print(analysis['Value Counts'])
    print("\n" + "="*50 + "\n")


Feature: TEAM_NAME
Unique Count: 30
Value Counts:
TEAM_NAME
Atlanta Hawks             808933
Indiana Pacers            768775
Golden State Warriors     752846
Washington Wizards        740414
Los Angeles Lakers        715769
Denver Nuggets            704216
Utah Jazz                 702849
San Antonio Spurs         699295
Houston Rockets           695282
Philadelphia 76ers        685368
Sacramento Kings          675514
Toronto Raptors           675109
Minnesota Timberwolves    672308
Portland Trail Blazers    669300
Miami Heat                669245
Boston Celtics            667801
Oklahoma City Thunder     667060
New York Knicks           664969
Detroit Pistons           657963
Charlotte Hornets         656016
Brooklyn Nets             651374
Cleveland Cavaliers       650237
Phoenix Suns              639063
Milwaukee Bucks           625563
New Orleans Pelicans      618640
Dallas Mavericks          617229
LA Clippers               613144
Orlando Magic             610981
Chicago Bulls   

In [ ]:
Low_cardinality_columns = ['SHOT_TYPE', 'BASIC_ZONE', 'ZONE_NAME', 'ZONE_RANGE', 'POSITION_GROUP']


In [ ]:
# Print columns before encoding transformations
print("Columns before encoding transformations:")
print(df.columns.tolist())

# Save the original columns for comparison after encoding
original_columns = df.columns.tolist()

# Perform encoding transformations
df_encoded = one_hot_encode(df_encoded, Low_cardinality_columns, drop_first= False)
df_encoded = frequency_encode(df_encoded, columns=['ACTION_TYPE'])

# Check for missing columns after transformations
missing_columns = [col for col in original_columns if col not in df_encoded.columns]
if missing_columns:
    print(f"Warning: These columns are missing after encoding: {missing_columns}")
else:
    print("All columns are retained after encoding.")


Columns before encoding transformations:
['TEAM_ID', 'TEAM_NAME', 'PLAYER_ID', 'PLAYER_NAME', 'POSITION_GROUP', 'POSITION', 'GAME_DATE', 'GAME_ID', 'HOME_TEAM', 'AWAY_TEAM', 'SHOT_MADE', 'ACTION_TYPE', 'SHOT_TYPE', 'BASIC_ZONE', 'ZONE_NAME', 'ZONE_RANGE', 'LOC_X', 'LOC_Y', 'SHOT_DISTANCE', 'QUARTER', 'MINS_LEFT', 'SECS_LEFT', 'SPD', 'DIST', 'ORBC', 'DRBC', 'TCHS', 'SAST', 'AST', 'FGM', 'FGA', 'DFGM', 'DFGA', 'DFG_PCT', 'SHOT_CLOCK_RANGE', 'SCORE', 'SCOREMARGIN', 'EFG_PCT', 'PERSON1TYPE', 'PERSON2TYPE', 'PERSON3TYPE', 'LOC_X_BIN', 'LOC_Y_BIN', 'SHOT_DISTANCE_BIN', 'ACTION_TYPE_modified', 'game_time', 'time_left_in_quarter', 'time_left_in_game', 'last_shot_of_quarter', 'shot_clock_status', 'is_clutch']


In [ ]:
Remaining_columns = ['shot_clock_status','quarter_progress']
df_encoded = one_hot_encode(df_encoded, Remaining_columns, drop_first= False)

In [ ]:
df_encoded.shape

(20134694, 96)

In [ ]:
df_encoded.columns

Index(['TEAM_ID', 'TEAM_NAME', 'PLAYER_ID', 'PLAYER_NAME', 'POSITION',
       'GAME_DATE', 'GAME_ID', 'HOME_TEAM', 'AWAY_TEAM', 'SHOT_MADE',
       'ACTION_TYPE', 'LOC_X', 'LOC_Y', 'SHOT_DISTANCE', 'QUARTER',
       'MINS_LEFT', 'SECS_LEFT', 'SPD', 'DIST', 'ORBC', 'DRBC', 'TCHS', 'SAST',
       'AST', 'FGM', 'FGA', 'DFGM', 'DFGA', 'DFG_PCT', 'SHOT_CLOCK_RANGE',
       'SCORE', 'SCOREMARGIN', 'EFG_PCT', 'PERSON1TYPE', 'PERSON2TYPE',
       'PERSON3TYPE', 'LOC_X_BIN', 'LOC_Y_BIN', 'SHOT_DISTANCE_BIN',
       'ACTION_TYPE_modified', 'game_time', 'time_left_in_quarter',
       'time_left_in_game', 'last_shot_of_quarter', 'is_clutch',
       'rolling_efg_pct', 'rolling_score_margin', 'last_30_seconds',
       'shots_in_last_30_seconds', 'SHOT_ATTEMPT', 'zone_combination',
       'zone_combination_attempts', 'zone_combination_made',
       'zone_combination_success_rate', 'BASIC_ZONE_freq', 'zone_dfg%', 'DFG%',
       'zone_fg%', 'clutch_efg_pct', 'clutch_dfg_pct', 'is_hot_zone',
       'sco

In [ ]:

top_positions = df['POSITION'].value_counts().nlargest(6).index
df_encoded['POSITION_modified'] = df_encoded['POSITION'].apply(lambda x: x if x in top_positions else 'Other')
position_freq_encoding = df_encoded['POSITION_modified'].value_counts().to_dict()
df_encoded['POSITION_freq'] = df_encoded['POSITION_modified'].map(position_freq_encoding)
df_encoded.shape


(20134694, 98)

In [ ]:
df_encoded.columns




Index(['TEAM_ID', 'TEAM_NAME', 'PLAYER_ID', 'PLAYER_NAME', 'POSITION',
       'GAME_DATE', 'GAME_ID', 'HOME_TEAM', 'AWAY_TEAM', 'SHOT_MADE',
       'ACTION_TYPE', 'LOC_X', 'LOC_Y', 'SHOT_DISTANCE', 'QUARTER',
       'MINS_LEFT', 'SECS_LEFT', 'SPD', 'DIST', 'ORBC', 'DRBC', 'TCHS', 'SAST',
       'AST', 'FGM', 'FGA', 'DFGM', 'DFGA', 'DFG_PCT', 'SHOT_CLOCK_RANGE',
       'SCORE', 'SCOREMARGIN', 'EFG_PCT', 'PERSON1TYPE', 'PERSON2TYPE',
       'PERSON3TYPE', 'LOC_X_BIN', 'LOC_Y_BIN', 'SHOT_DISTANCE_BIN',
       'ACTION_TYPE_modified', 'game_time', 'time_left_in_quarter',
       'time_left_in_game', 'last_shot_of_quarter', 'is_clutch',
       'rolling_efg_pct', 'rolling_score_margin', 'last_30_seconds',
       'shots_in_last_30_seconds', 'SHOT_ATTEMPT', 'zone_combination',
       'zone_combination_attempts', 'zone_combination_made',
       'zone_combination_success_rate', 'BASIC_ZONE_freq', 'zone_dfg%', 'DFG%',
       'zone_fg%', 'clutch_efg_pct', 'clutch_dfg_pct', 'is_hot_zone',
       'sco

# Normalization

In [ ]:
features_to_analyze = [
    'LOC_X', 'LOC_Y', 'SHOT_DISTANCE', 'SPD', 'DIST',
    'ORBC', 'DRBC', 'TCHS', 'SAST', 'AST',
    'FGM', 'FGA', 'DFGM', 'DFGA', 'DFG_PCT', 'EFG_PCT',
    'SCOREMARGIN'
]

skewness = df_encoded[features_to_analyze].skew()
print("Skewness of Features:\n", skewness)


Skewness of Features:
 LOC_X            0.018248
LOC_Y            0.829714
SHOT_DISTANCE    0.160786
SPD             -0.113624
DIST            -0.609528
ORBC             1.499096
DRBC             0.983134
TCHS             0.329478
SAST             1.776295
AST              1.087790
FGM              2.462394
FGA              2.422308
DFGM             1.113227
DFGA             1.325714
DFG_PCT         -0.522054
EFG_PCT          0.285988
SCOREMARGIN      0.284261
dtype: float64


In [ ]:

highly_skewed_features = skewness[abs(skewness) > 1].index.tolist()
print("Highly Skewed Features:\n", highly_skewed_features)

pt = PowerTransformer(method='yeo-johnson', standardize=False)
df_encoded[highly_skewed_features] = pt.fit_transform(df_encoded[highly_skewed_features])

skewness_after = df_encoded[highly_skewed_features].skew()
print("Skewness After Transformation:\n", skewness_after)


Highly Skewed Features:
 ['ORBC', 'SAST', 'AST', 'FGM', 'FGA', 'DFGM', 'DFGA']


/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


Skewness After Transformation:
 ORBC   -0.008148
SAST    0.563825
AST    -0.030914
FGM    -0.004749
FGA    -0.008299
DFGM   -0.019034
DFGA   -0.016048
dtype: float32


In [ ]:
categorical_columns = ['ACTION_TYPE', 'SCORE']
for col in categorical_columns:
    df_encoded[col] = df_encoded[col].astype('category')

df_encoded['SCOREMARGIN'] = df_encoded['SCOREMARGIN'].astype('float32')
df_encoded['time_left_in_game'] = df_encoded['time_left_in_game'].astype('int32')
print(df_encoded.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20134694 entries, 0 to 20134693
Data columns (total 98 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   TEAM_ID                           int32   
 1   TEAM_NAME                         object  
 2   PLAYER_ID                         int32   
 3   PLAYER_NAME                       object  
 4   POSITION                          object  
 5   GAME_DATE                         object  
 6   GAME_ID                           int32   
 7   HOME_TEAM                         object  
 8   AWAY_TEAM                         object  
 9   SHOT_MADE                         bool    
 10  ACTION_TYPE                       category
 11  LOC_X                             float32 
 12  LOC_Y                             float32 
 13  SHOT_DISTANCE                     float32 
 14  QUARTER                           int8    
 15  MINS_LEFT                         int16   
 16  SECS_LEFT       

In [ ]:
df_encoded.isnull().sum()

,0
TEAM_ID,0
TEAM_NAME,0
PLAYER_ID,0
PLAYER_NAME,0
POSITION,0
GAME_DATE,0
GAME_ID,0
HOME_TEAM,0
AWAY_TEAM,0
SHOT_MADE,0


In [ ]:
df_encoded.to_parquet('/content/drive/MyDrive/Encoded_data.parquet', index=False, compression='gzip')
